<h1>other one <h1>

In [1]:
!pip install  ortools

  Using cached ortools-7.8.7959-cp37-cp37m-win_amd64.whl (47.8 MB)


In [2]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [
        [0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
        [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
        [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
        [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
        [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
        [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
        [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
        [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
        [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
        [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
        [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
        [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
        [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0],
    ]  # yapf: disable
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data
from ortools.constraint_solver import pywrapcp

data = create_data_model()
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)
solution = routing.SolveWithParameters(search_parameters)
if solution:
    print_solution(manager, routing, solution)


In [2]:
"""Simple travelling salesman problem between cities."""

from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp



def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [
        [0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
        [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
        [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
        [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
        [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
        [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
        [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
        [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
        [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
        [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
        [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
        [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
        [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0],
    ]  # yapf: disable
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data


def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(manager, routing, solution)


if __name__ == '__main__':
    main()

ImportError: DLL load failed: Le module spécifié est introuvable.

In [89]:
import numpy as np
import mysql.connector
from mysql.connector import Error


def get_coordinates(adress):
    records=[]
    try:
        connection = mysql.connector.connect(host='localhost',
                                         database='optimatets',
                                         user='root',
                                         password='')
        reqet = "SELECT lat,lng FROM places  where city='"+adress+"' AND name='city';"
        cursor = connection.cursor()
        cursor.execute(reqet)
        records.append(cursor.fetchall())
    except Error as e:
        print("Error reading data from MySQL table", e)
    finally:
        if (connection.is_connected()):
            connection.close()
            cursor.close()
            print("MySQL connection is closed")
    
    return(records[0][0])

def fill_cordinates_matrix(adress):
    lat=[]
    lng=[]
    for x in adress:
        lat.append(get_coordinates(adress[x])[0][0])
        lng.append(get_coordinates(adress[x])[0][1])
    return [lat,lng]
        
    
def fill_cordinates(adress):
    corr=[]
    for x in adress:
        corr.append(get_coordinates(adress[x]))
        
    return corr
    
        
def get_matrix_distace(adress):
    from scipy.spatial import distance
    coordinates=fill_cordinates(adress)
    matrix=distance.cdist(coordinates, coordinates, 'euclidean')
    return matrix





In [90]:
from flask import Flask, request, make_response
import json
app = Flask(__name__)
@app.route('/', methods=['POST'])
def index():
    
    d = request.form.to_dict() # data recived from form
   
    data=make_response(json.dumps({"name":"yes"})) # transform final result to json
    resp=data
    resp.status_code = 200
    resp.headers['Access-Control-Allow-Origin'] = '*' # configiration 

    print(d)
    print(type(d))
    print(get_matrix_distace(d))
    print(type(get_matrix_distace(d)))
    return resp


    
    
    
if __name__ == "__main__":
    app.run()
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[2020-08-23 02:03:13,722] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "C:\Users\meher\anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\meher\anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\meher\anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\meher\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\meher\anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\meher\anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-90-7c9

{}
<class 'dict'>


127.0.0.1 - - [23/Aug/2020 02:03:31] "POST / HTTP/1.1" 200 -


{'adress0': 'sousse', 'adress1': 'tunis', 'adress2': 'sfax', 'adress3': 'mahdia'}
<class 'dict'>
MySQL connection is closed
MySQL connection is closed
MySQL connection is closed
MySQL connection is closed
[[0.         1.08574182 1.09178475 0.53258195]
 [1.08574182 0.         2.14789359 1.5772448 ]
 [1.09178475 2.14789359 0.         0.82157922]
 [0.53258195 1.5772448  0.82157922 0.        ]]


[2020-08-23 02:05:23,893] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "C:\Users\meher\anaconda3\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\meher\anaconda3\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\meher\anaconda3\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\meher\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\meher\anaconda3\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\meher\anaconda3\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-90-7c9b5cc7f228>", line 32, in index
    print(get_matrix_distace(

{'adress0': 'el kef', 'adress1': 'sousse', 'adress2': 'kasserine', 'adress3': 'sfax', 'adress4': 'mednine'}
<class 'dict'>
MySQL connection is closed
MySQL connection is closed
MySQL connection is closed
MySQL connection is closed
MySQL connection is closed


127.0.0.1 - - [23/Aug/2020 02:06:45] "POST / HTTP/1.1" 200 -


{'adress0': 'sfax', 'adress1': 'sousse', 'adress2': 'kasserine', 'adress3': 'sfax', 'adress4': 'medenine', 'adress5': 'gafsa'}
<class 'dict'>
MySQL connection is closed
MySQL connection is closed
MySQL connection is closed
MySQL connection is closed
MySQL connection is closed
MySQL connection is closed
[[0.         1.09178475 1.97060827 0.         1.40893134 2.00117294]
 [1.09178475 0.         1.91688943 1.09178475 2.47399727 2.32252   ]
 [1.97060827 1.91688943 0.         1.97060827 2.46402291 0.74444223]
 [0.         1.09178475 1.97060827 0.         1.40893134 2.00117294]
 [1.40893134 2.47399727 2.46402291 1.40893134 0.         2.02684261]
 [2.00117294 2.32252    0.74444223 2.00117294 2.02684261 0.        ]]


In [91]:
d={'adress0': 'sfax', 'adress1': 'sousse', 'adress2': 'kasserine', 'adress3': 'sfax', 'adress4': 'medenine', 'adress5': 'gafsa'}
print(type(get_matrix_distace(d)))

MySQL connection is closed
MySQL connection is closed
MySQL connection is closed
MySQL connection is closed
MySQL connection is closed
MySQL connection is closed
<class 'numpy.ndarray'>


In [96]:
type(list(d.values()))

list

In [97]:
list(d.values())

['sfax', 'sousse', 'kasserine', 'sfax', 'medenine', 'gafsa']